In [1]:
import random
import numpy as np
import math
from nnfunc import sigmoid, tanh, rand_arr

In [2]:
np.random.seed(0)

In [3]:
class LstmParam:
    def __init__(self, mem_cell_ct, x_dim):
        self.mem_cell_ct = mem_cell_ct
        self.x_dim = x_dim
        concat_len = x_dim + mem_cell_ct
        # ваги
        self.wg = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        self.wi = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len) 
        self.wf = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        self.wo = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        # зміщення
        self.bg = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bi = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bf = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bo = rand_arr(-0.1, 0.1, mem_cell_ct) 
        # різниці/дельти (похідні функції втрат)
        self.wg_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wi_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wf_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wo_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.bg_diff = np.zeros(mem_cell_ct) 
        self.bi_diff = np.zeros(mem_cell_ct) 
        self.bf_diff = np.zeros(mem_cell_ct) 
        self.bo_diff = np.zeros(mem_cell_ct) 

    def apply_diff(self, lr = 1):
        self.wg -= lr * self.wg_diff
        self.wi -= lr * self.wi_diff
        self.wf -= lr * self.wf_diff
        self.wo -= lr * self.wo_diff
        self.bg -= lr * self.bg_diff
        self.bi -= lr * self.bi_diff
        self.bf -= lr * self.bf_diff
        self.bo -= lr * self.bo_diff
        # скидування значень дельт
        self.wg_diff = np.zeros_like(self.wg)
        self.wi_diff = np.zeros_like(self.wi) 
        self.wf_diff = np.zeros_like(self.wf) 
        self.wo_diff = np.zeros_like(self.wo) 
        self.bg_diff = np.zeros_like(self.bg)
        self.bi_diff = np.zeros_like(self.bi) 
        self.bf_diff = np.zeros_like(self.bf) 
        self.bo_diff = np.zeros_like(self.bo) 

In [4]:
class LstmState:
    def __init__(self, mem_cell_ct, x_dim):
        self.g = np.zeros(mem_cell_ct)
        self.i = np.zeros(mem_cell_ct)
        self.f = np.zeros(mem_cell_ct)
        self.o = np.zeros(mem_cell_ct)
        self.s = np.zeros(mem_cell_ct)
        self.h = np.zeros(mem_cell_ct)
        self.bottom_diff_h = np.zeros_like(self.h)
        self.bottom_diff_s = np.zeros_like(self.s)
    
class LstmNode:
    def __init__(self, lstm_param, lstm_state):
        # збереження ваг та активацій
        self.state = lstm_state
        self.param = lstm_param
        # не рекурентні зв'язки
        self.xc = None

    def bottom_data_is(self, x, s_prev = None, h_prev = None):
        # якщо перша вершина мережі
        if s_prev is None: s_prev = np.zeros_like(self.state.s)
        if h_prev is None: h_prev = np.zeros_like(self.state.h)
        # зберігаємо дані для зворотнього ходу
        self.s_prev = s_prev
        self.h_prev = h_prev

        # з'єднуємо x(t) та h(t-1)
        xc = np.hstack((x,  h_prev))
        self.state.g = np.tanh(np.dot(self.param.wg, xc) + self.param.bg)
        self.state.i = sigmoid(np.dot(self.param.wi, xc) + self.param.bi)
        self.state.f = sigmoid(np.dot(self.param.wf, xc) + self.param.bf)
        self.state.o = sigmoid(np.dot(self.param.wo, xc) + self.param.bo)
        self.state.s = self.state.g * self.state.i + s_prev * self.state.f
        self.state.h = self.state.s * self.state.o

        self.xc = xc
    
    def top_diff_is(self, top_diff_h, top_diff_s):
        # ініціалізація дельт
        ds = self.state.o * top_diff_h + top_diff_s
        do = self.state.s * top_diff_h
        di = self.state.g * ds
        dg = self.state.i * ds
        df = self.s_prev * ds

        # різниці активацій
        di_input = sigmoid(self.state.i, True) * di 
        df_input = sigmoid(self.state.f, True) * df 
        do_input = sigmoid(self.state.o, True) * do 
        dg_input = tanh(self.state.g, True) * dg

        # різниці входів
        self.param.wi_diff += np.outer(di_input, self.xc)
        self.param.wf_diff += np.outer(df_input, self.xc)
        self.param.wo_diff += np.outer(do_input, self.xc)
        self.param.wg_diff += np.outer(dg_input, self.xc)
        self.param.bi_diff += di_input
        self.param.bf_diff += df_input       
        self.param.bo_diff += do_input
        self.param.bg_diff += dg_input       

        # різниці наступних шарів
        dxc = np.zeros_like(self.xc)
        dxc += np.dot(self.param.wi.T, di_input)
        dxc += np.dot(self.param.wf.T, df_input)
        dxc += np.dot(self.param.wo.T, do_input)
        dxc += np.dot(self.param.wg.T, dg_input)

        self.state.bottom_diff_s = ds * self.state.f
        self.state.bottom_diff_h = dxc[self.param.x_dim:]

In [5]:
class LstmNetwork():
    def __init__(self, lstm_param):
        self.lstm_param = lstm_param
        self.lstm_node_list = []
        # вхідна послідовність
        self.x_list = []

    def y_list_is(self, y_list, loss_layer):
        assert len(y_list) == len(self.x_list)
        idx = len(self.x_list) - 1

        loss = loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
        diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
        # s не враховується у втратах дл h(t+1), оскільки тут воно 0
        diff_s = np.zeros(self.lstm_param.mem_cell_ct)
        self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
        idx -= 1

        # передача дельт від наступних шарів до попередніх
        # та поширення помилки через diff_s
        while idx >= 0:
            loss += loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h += self.lstm_node_list[idx + 1].state.bottom_diff_h
            diff_s = self.lstm_node_list[idx + 1].state.bottom_diff_s
            self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
            idx -= 1 

        return loss

    def x_list_clear(self):
        self.x_list = []

    def x_list_add(self, x):
        self.x_list.append(x)
        if len(self.x_list) > len(self.lstm_node_list):
            # додаємо нову LstmNode, тобто новий стан
            lstm_state = LstmState(self.lstm_param.mem_cell_ct, self.lstm_param.x_dim)
            self.lstm_node_list.append(LstmNode(self.lstm_param, lstm_state))

        # беремо останній ввід
        idx = len(self.x_list) - 1
        if idx == 0:
            self.lstm_node_list[idx].bottom_data_is(x)
        else:
            s_prev = self.lstm_node_list[idx - 1].state.s
            h_prev = self.lstm_node_list[idx - 1].state.h
            self.lstm_node_list[idx].bottom_data_is(x, s_prev, h_prev)

In [6]:
class LossLayer:
    @classmethod
    def loss(self, pred, label):
        return (pred[0] - label) ** 2

    @classmethod
    def bottom_diff(self, pred, label):
        diff = np.zeros_like(pred)
        diff[0] = 2 * (pred[0] - label)
        return diff

In [7]:
mem_cell_ct = 100
x_dim = 50
lstm_param = LstmParam(mem_cell_ct, x_dim)
lstm_net = LstmNetwork(lstm_param)
y_list = [-0.5, 0.2, 0.1, -0.5]
# ініціалізація ваг для входів
input_val_arr = [np.random.random(x_dim) for _ in y_list]

for cur_iter in range(250):
    print("Ітерація", "%2s" % str(cur_iter), end=": ")
    for ind in range(len(y_list)):
        lstm_net.x_list_add(input_val_arr[ind])

    print("y_pred = [" +
            ", ".join(["% 2.5f" % lstm_net.lstm_node_list[ind].state.h[0] for ind in range(len(y_list))]) +
            "]", end=", ")

    loss = lstm_net.y_list_is(y_list, LossLayer)
    print("втрати:", "%.3e" % loss)
    lstm_param.apply_diff(lr=0.1)
    lstm_net.x_list_clear()

Ітерація  0: y_pred = [ 0.07300,  0.08913,  0.12884,  0.12905], втрати: 7.372e-01
Ітерація  1: y_pred = [-0.09211, -0.12664, -0.15126, -0.16966], втрати: 4.453e-01
Ітерація  2: y_pred = [-0.10237, -0.12602, -0.15080, -0.18605], втрати: 4.259e-01
Ітерація  3: y_pred = [-0.11175, -0.12311, -0.14770, -0.20090], втрати: 4.059e-01
Ітерація  4: y_pred = [-0.12111, -0.11860, -0.14291, -0.21525], втрати: 3.851e-01
Ітерація  5: y_pred = [-0.13091, -0.11256, -0.13658, -0.22942], втрати: 3.631e-01
Ітерація  6: y_pred = [-0.14150, -0.10484, -0.12857, -0.24350], втрати: 3.395e-01
Ітерація  7: y_pred = [-0.15327, -0.09523, -0.11867, -0.25760], втрати: 3.140e-01
Ітерація  8: y_pred = [-0.16667, -0.08356, -0.10670, -0.27195], втрати: 2.863e-01
Ітерація  9: y_pred = [-0.18217, -0.06977, -0.09261, -0.28690], втрати: 2.563e-01
Ітерація 10: y_pred = [-0.20016, -0.05386, -0.07640, -0.30284], втрати: 2.243e-01
Ітерація 11: y_pred = [-0.22081, -0.03593, -0.05820, -0.31998], втрати: 1.910e-01
Ітерація 12: y_p